# Demonstration of anomaly detection with CVAE using DASHlink data

**Author: Milad Memarzadeh (milad.memarzadeh@nasa.gov)**


# Demonstration of anomaly detection with CVAE using DASHlink data

**Author: Milad Memarzadeh (milad.memarzadeh@nasa.gov)**


# Import Libraries

In [126]:
import os
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support, roc_curve, precision_recall_curve, average_precision_score
from sklearn.utils import shuffle
#from source.modelsCondVAE import *
from source.utilsCondVAEs5 import *
from itertools import combinations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [127]:
train_learning_rate = 1e-4 #1e-3
train_num_epochs = 10

latent_dim = 10
batch_size = 64
window_size = 10


# Load and preprocess data

In [128]:
if os.name == 'nt':
    data_dir = 'C:/Users/jed95/Documents/GitHub/anomaly_detection/dataset/yahoo_s5/A2Benchmark/'
else:
    data_dir = '/home/adlink3/Downloads/yahoo_s5/A2Benchmark/'
save_dir="./CompressAE/A2/"
file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]


df_list = []
for file in file_list:
    df = pd.read_csv(file)
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
print("Data shape:", data.shape)


Data shape: (142100, 3)


In [129]:
print("Missing values:", data.isnull().sum())
data.dropna(inplace=True)

Missing values: timestamp     0
value         0
is_anomaly    0
dtype: int64


In [130]:
scaler = MinMaxScaler()
data['value'] = scaler.fit_transform(data['value'].values.reshape(-1, 1))


# Create Sequences

In [131]:
def create_sequences(values, labels, window_size, step=1):
    sequences = []
    seq_labels = []
    for i in range(0, len(values) - window_size + 1, step):
        seq = values[i:i + window_size]
        # If any label in the sequence is anomalous (e.g., 1), set the sequence label as anomalous
        label = 1 if any(labels[i:i + window_size]) else 0
        sequences.append(seq)
        seq_labels.append(label)
    return np.array(sequences), np.array(seq_labels)


sequences, seq_labels = create_sequences(data['value'].values, data['is_anomaly'].values, window_size=window_size)
print("Sequences shape:", sequences.shape)
print("Sequence labels shape:", seq_labels.shape)



Sequences shape: (142091, 10)
Sequence labels shape: (142091,)


# Split data into labeled and unlabeled sets

In [132]:

normal_indices = np.where(seq_labels == 0)[0]
anomalous_indices = np.where(seq_labels == 1)[0]

normal_sequences = sequences[normal_indices]
normal_labels = seq_labels[normal_indices]

anomalous_sequences = sequences[anomalous_indices]
anomalous_labels = seq_labels[anomalous_indices]
print(normal_sequences.shape)
print(anomalous_sequences.shape)


(139825, 10)
(2266, 10)


In [133]:
X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(
    normal_sequences, normal_labels, test_size=0.6, random_state=42, stratify=normal_labels)
X_train_normal, X_val_normal, y_train_normal, y_val_normal = train_test_split(
    X_train_normal, y_train_normal, test_size=0.5, random_state=42, stratify=y_train_normal)


X_train_anomalous, X_test_anomalous, y_train_anomalous, y_test_anomalous = train_test_split(
    anomalous_sequences, anomalous_labels, test_size=0.6, random_state=42, stratify=anomalous_labels)
X_train_anomalous, X_val_anomalous, y_train_anomalous, y_val_anomalous = train_test_split(
    X_train_anomalous, y_train_anomalous, test_size=0.5, random_state=42, stratify=y_train_anomalous)


In [134]:
X_train = np.concatenate([X_train_normal, X_train_anomalous], axis=0)
y_train = np.concatenate([y_train_normal, y_train_anomalous], axis=0)

X_val = np.concatenate([X_val_normal, X_val_anomalous], axis=0)
y_val = np.concatenate([y_val_normal, y_val_anomalous], axis=0)

X_test = np.concatenate([X_test_normal, X_test_anomalous], axis=0)
y_test = np.concatenate([y_test_normal, y_test_anomalous], axis=0)
print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_val.shape, y_val.shape)
print("Test data shape:", X_test.shape, y_test.shape)


Training data shape: (28418, 10) (28418,)
Validation data shape: (28418, 10) (28418,)
Test data shape: (85255, 10) (85255,)


In [135]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_val, y_val = shuffle(X_val, y_val, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)


In [ ]:
np.save(save_dir+'X_train.npy', X_train)
np.save(save_dir+'y_train.npy', y_train)
np.save(save_dir+'X_val.npy', X_val)
np.save(save_dir+'y_val.npy', y_val)
np.save(save_dir+'X_test.npy', X_test)
np.save(save_dir+'y_test.npy', y_test)

X_train_size = os.path.getsize(save_dir+'X_train.npy')
X_val_size = os.path.getsize(save_dir+'X_val.npy')
X_test_size = os.path.getsize(save_dir+'X_test.npy')

X_train_tensor shape: torch.Size([28418, 1, 10])
y_train_tensor shape: torch.Size([28418])
X_val_tensor shape: torch.Size([28418, 1, 10])
y_val_tensor shape: torch.Size([28418])
X_test_tensor shape: torch.Size([85255, 1, 10])
y_test_tensor shape: torch.Size([85255])


# Convert Data to Tensors

In [ ]:
X_train_tensor = torch.tensor(X_train).unsqueeze(1).float()
y_train_tensor = torch.tensor(y_train).long()

X_val_tensor = torch.tensor(X_val).unsqueeze(1).float()
y_val_tensor = torch.tensor(y_val).long()

X_test_tensor = torch.tensor(X_test).unsqueeze(1).float()
y_test_tensor = torch.tensor(y_test).long()
print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)
print("X_val_tensor shape:", X_val_tensor.shape)
print("y_val_tensor shape:", y_val_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)
print("y_test_tensor shape:", y_test_tensor.shape)


# Create Data Loaders for Labeled and Unlabeled Data

In [138]:

#train_dataset = TimeSeriesDataset(X_train_tensor, y_train_tensor)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) #TODO adjust Shuffle training data? 
#full_train_dataset = TimeSeriesDataset(X_train_tensor, y_train_tensor)
#
#val_dataset = TimeSeriesDataset(X_val_tensor, y_val_tensor)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#
#
#test_dataset = TimeSeriesDataset(X_test_tensor, y_test_tensor)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#
X_train_dataset = torch.utils.data.TensorDataset(X_train_tensor)
X_train_loader = torch.utils.data.DataLoader(X_train_dataset, batch_size=batch_size, shuffle=False)

In [139]:
class Conv1dAutoencoder(nn.Module):
    def __init__(self, input_length, latent_dim):
        super(Conv1dAutoencoder, self).__init__()
        self.input_length = input_length
        self.latent_dim = latent_dim

        # Encoder: Conv1d downsampling
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv1d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(True)
        )
        # infer flattened size
        with torch.no_grad():
            dummy = torch.zeros(1, 1, self.input_length)
            enc = self.encoder(dummy)
            c, l = enc.shape[1], enc.shape[2]
            self._flattened_size = c * l

        # Bottleneck
        self.fc1 = nn.Linear(self._flattened_size, self.latent_dim)
        self.fc2 = nn.Linear(self.latent_dim, self._flattened_size)

        # Decoder: ConvTranspose1d upsampling
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (c, l)),
            nn.ConvTranspose1d(c, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        return self.fc1(x)

    def decode(self, z):
        x = self.fc2(z)
        x = self.decoder(x)
        return x[..., :self.input_length]

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z), z


In [140]:
model = Conv1dAutoencoder(input_length=window_size, latent_dim=latent_dim).to(device)
opt = optim.Adam(model.parameters(), lr=train_learning_rate)
loss_fn = nn.MSELoss()

model.train()
for ep in range(train_num_epochs):
    tot=0
    for bx, in X_train_loader:
        bx = bx.to(device)
        recon, _ = model(bx)
        loss = loss_fn(recon, bx)
        opt.zero_grad(); loss.backward(); opt.step()
        tot+=loss.item()
    print(f'Epoch {ep+1} Loss {tot/len(X_train_loader):.6f}')

Epoch 1 Loss 0.133404
Epoch 2 Loss 0.041641
Epoch 3 Loss 0.008874
Epoch 4 Loss 0.004397
Epoch 5 Loss 0.001286
Epoch 6 Loss 0.000956
Epoch 7 Loss 0.000897
Epoch 8 Loss 0.000843
Epoch 9 Loss 0.000747
Epoch 10 Loss 0.000502


In [141]:
model.eval()
#print(summary(
#model,
#input_size=(32, 1, window_size),
#))

Conv1dAutoencoder(
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(16, 8, kernel_size=(3,), stride=(2,), padding=(1,))
    (3): ReLU(inplace=True)
  )
  (fc1): Linear(in_features=24, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=24, bias=True)
  (decoder): Sequential(
    (0): Unflatten(dim=1, unflattened_size=(8, 3))
    (1): ConvTranspose1d(8, 16, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,))
    (2): ReLU(inplace=True)
    (3): ConvTranspose1d(16, 1, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,))
    (4): Sigmoid()
  )
)

In [142]:
with torch.no_grad():
    recon1, z_train = model(X_train_tensor.to(device))
    recon2, z_val = model(X_val_tensor.to(device))
    recon3, z_test = model(X_test_tensor.to(device))
np.save(save_dir+'z_train.npy', z_train.cpu().numpy())
np.save(save_dir+'z_val.npy', z_val.cpu().numpy())
np.save(save_dir+'z_test.npy', z_test.cpu().numpy())
#dfz = pd.DataFrame(z, columns=[f'z{i}' for i in range(z.shape[1])])
#dfz['label'] = y_train
#dfz.to_csv('compressed_with_labels.csv', index=False)
# sizes
z_train_size = os.path.getsize(save_dir+'z_train.npy')
z_val_size = os.path.getsize(save_dir+'z_val.npy')
z_test_size = os.path.getsize(save_dir+'z_test.npy')

In [ ]:
train_ratio = z_train_size / X_train_size if X_train_size else float('nan')
print(f"Uncompressed: {X_train_size} bytes\nCompressed: {z_train_size} bytes\nRatio: {train_ratio:.3f}\n")
val_ratio = z_val_size / X_val_size if X_val_size else float('nan')
print(f"Uncompressed: {X_val_size} bytes\nCompressed: {z_val_size} bytes\nRatio: {val_ratio:.3f}\n")
test_ratio = z_test_size / X_test_size if X_test_size else float('nan')
print(f"Uncompressed: {X_test_size} bytes\nCompressed: {z_test_size} bytes\nRatio: {test_ratio:.3f}\n")


Uncompressed: 1136848 bytes
Compressed: 1136848 bytes
Ratio: 1.000

Uncompressed: 1136848 bytes
Compressed: 1136848 bytes
Ratio: 1.000

Uncompressed: 3410328 bytes
Compressed: 3410328 bytes
Ratio: 1.000



# Instantiate and Train the Conditional VAE

In [144]:
def evaluate_model(model, data_loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            x_rec, class_logits = model(data)
            
            # Compute classification loss
            loss = criterion(class_logits, target)
            total_loss += loss.item() * data.size(0)

            # Compute accuracy
            preds = class_logits.argmax(dim=1)
            total_correct += (preds == target).sum().item()
            total_samples += data.size(0)

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples

    return avg_loss, avg_acc


In [145]:
def plot_training_loss(model_name, save_dir):
    training_traj = np.load(save_dir+model_name+"_training_loss.npz")

    total_loss = training_traj['training_total_loss']
    rec_loss = training_traj['training_rec_loss']
    kl_loss = training_traj['training_kl_loss']
    class_loss = training_traj['training_class_loss']
    plt.figure(figsize=(16, 16))
    plt.suptitle(model_name, fontsize=16)
    plt.subplot(2, 2, 1)
    plt.title("Total Loss", fontsize=12)
    plt.plot(range(len(total_loss)), total_loss)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.subplot(2, 2, 2)
    plt.title("Reconstruction Loss", fontsize=12)
    plt.plot(range(len(total_loss)), rec_loss)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.subplot(2, 2, 3)
    plt.title("KL Loss", fontsize=12)
    plt.plot(range(len(total_loss)), kl_loss)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.subplot(2, 2, 4)
    plt.title("Classification Loss", fontsize=12)
    plt.plot(range(len(total_loss)), class_loss)
    plt.tick_params(axis='both', which='major', labelsize=14)
    
#for fold in range(k_folds):
#    plot_training_loss("CondVAE_model"+str(fold+1), save_dir)

# Initialize 3 new models for fine-tuning

# Verify the 3 new models

# Evaluate the Model and Detect Anomalies

In [ ]:
def compute_anomaly_scores(model, data_loader):
    model.eval()
    rec_errors = []
    anomaly_probs = []
    true_labels = []
    predictions = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.unsqueeze(1).to(device)
            y_batch = y_batch.to(device)

            x_rec, class_logits = model(X_batch)
            y_probs = F.softmax(class_logits, dim=1)

            # Reconstruction error
            rec_error = torch.mean((X_batch - x_rec) ** 2, dim=[1, 2])

            # Classification probability for anomaly class
            anomaly_prob = y_probs[:, 1]

            # Combine scores
            #anomaly_score = rec_error * anomaly_prob
            rec_errors.extend(rec_error.cpu().numpy())
            anomaly_probs.extend(anomaly_prob.cpu().numpy())
            true_labels.extend(y_batch.cpu().numpy())
            predictions.extend(torch.argmax(class_logits, dim=1).cpu().numpy())

    return np.array(rec_errors), np.array(anomaly_probs), np.array(true_labels), np.array(predictions)



NameError: name 'train_loader' is not defined

In [ ]:

# Scatter Plot
plt.figure(figsize=(8, 6))
plt.scatter(range(len(val_anomaly_probs)), val_anomaly_probs, alpha=0.7, c=(val_true_labels!=val_predictions), cmap='coolwarm')
plt.title("Scatter Plot of Validation Anomaly Scores")
plt.xlabel("Index")
plt.ylabel("Anomaly Score")
plt.show()

# Scatter Plot
plt.figure(figsize=(8, 6))
plt.scatter(val_rec_errors, val_anomaly_probs, alpha=0.7, c=(val_true_labels==val_predictions), cmap='coolwarm')
plt.title("Scatter Plot of Validation Anomaly Scores with Reconstruction Errors")
plt.xlabel("Index")
plt.ylabel("Anomaly Score")
plt.show()

# Box Plot
plt.figure(figsize=(8, 6))
plt.boxplot(val_anomaly_probs, vert=False)
plt.title("Box Plot of Validation Anomaly Scores")
plt.xlabel("Anomaly Score")
plt.show()

# Histogram
plt.figure(figsize=(8, 6))
plt.hist(val_anomaly_probs, bins=20, alpha=0.7, edgecolor='black')
plt.title("Histogram of Validation Anomaly Scores")
plt.xlabel("Anomaly Score")
plt.ylabel("Frequency")
plt.show()


In [ ]:

# Scatter Plot
plt.figure(figsize=(8, 6))
plt.scatter(range(len(val_rec_errors)), val_rec_errors, alpha=0.7, c=val_true_labels, cmap='coolwarm')
plt.title("Scatter Plot of Validation Anomaly Scores (true labels)")
plt.xlabel("Index")
plt.ylabel("Anomaly Score")
plt.show()
# Scatter Plot
plt.figure(figsize=(8, 6))
plt.scatter(range(len(val_rec_errors)), val_rec_errors, alpha=0.7, c=val_predictions, cmap='coolwarm')
plt.title("Scatter Plot of Validation Anomaly Scores (predictions)")
plt.xlabel("Index")
plt.ylabel("Anomaly Score")
plt.show()
# Scatter Plot
plt.figure(figsize=(8, 6))
plt.scatter(range(len(val_rec_errors)), val_rec_errors, alpha=0.7, c=val_anomaly_probs, cmap='coolwarm')
plt.title("Scatter Plot of Validation Anomaly Scores (anomalous probability)")
plt.xlabel("Index")
plt.ylabel("Anomaly Score")
plt.show()

# Box Plot
plt.figure(figsize=(8, 6))
plt.boxplot(val_rec_errors, vert=False)
plt.title("Box Plot of Validation Anomaly Scores")
plt.xlabel("Anomaly Score")
plt.show()

# Histogram
plt.figure(figsize=(8, 6))
plt.hist(val_rec_errors, bins=20, alpha=0.7, edgecolor='black')
plt.title("Histogram of Validation Anomaly Scores")
plt.xlabel("Anomaly Score")
plt.ylabel("Frequency")
plt.show()
